## Using TensorFlow to deal with iris data

### 1 Create a simple graph and run it:  $f(x,y) = x^2\cdot y + y + 2$

In [1]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [2]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
sess.run(f)

42

In [3]:
sess.close()

In [4]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
result

42

### 2 Classify iris data using Logistic Regression by TensorFlow

#### 2.1 Normal Equation: $\theta = (X^T\cdot X)^{-1}\cdot X^T\cdot y$

Note that label is a 1D array, but we need to reshape it to a column vector to compute theta.

In [5]:
import numpy as np
from sklearn.datasets import load_iris

dataset = load_iris()
feature = dataset.data
m, n = feature.data.shape
label = dataset.target

In [6]:
feature_add_bias = np.c_[np.ones((m, 1)), feature]
X = tf.constant(feature_add_bias, dtype=tf.float32, name="X")
y = tf.constant(label.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
error = tf.matmul(X, theta) - y
mse = tf.reduce_mean(tf.square(error), name="mse")

with tf.Session() as sess:
    theta_result = theta.eval()
    mse_result = mse.eval()
    
mse_result

0.046385087

In [7]:
theta_result

array([[ 0.19220488],
       [-0.10975894],
       [-0.04425819],
       [ 0.22700629],
       [ 0.609898  ]], dtype=float32)

#### 2.2 Using Batch Gradient Descent to fit iris data
Gradient: $ \nabla_{\theta}MSE(\theta) = \frac{2}{m}X^T\cdot(X\cdot\theta - y) $ <br>
Gradient Descent step: $ \theta^{(next step)} = \theta - \eta\cdot \nabla_{\theta}MSE(\theta) $

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_feature = scaler.fit_transform(feature)

In [9]:
np.random.seed(42)

X = tf.constant(feature_add_bias, dtype=tf.float32, name="X")
y = tf.constant(label.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0, seed=42), name="theta")
error = tf.matmul(X, theta) - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), tf.matmul(X, theta) - y)
training = tf.assign(theta, theta - 0.01 * gradients)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        sess.run(training)
    best_mse = mse.eval()
    theta_result = theta.eval()
    
best_mse

0.05250996

In [10]:
theta_result

array([[ 0.6631392 ],
       [-0.30217153],
       [ 0.03044995],
       [ 0.44131878],
       [ 0.29573858]], dtype=float32)

#### 2.3 Using TensorBoard for Data Visualization

In [11]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [12]:
mse_summary = tf.summary.scalar("MSE", mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [13]:
with tf.Session() as sess:
    sess.run(init)
    for iter in range(1000):
        if iter % 10 == 0:
            summary = sess.run(mse_summary)
            file_writer.add_summary(summary, iter)
        sess.run(training)
    best_mse = mse.eval()
    theta_result = theta.eval()

best_mse

0.05250996

#### 2.4 Retrain the Gradient Descent by optimazer

In [14]:
learning_rate = 0.001
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train = optimizer.minimize(mse)

In [15]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(1000):
        sess.run(training)
    best_mse = mse.eval()
    theta_result = theta.eval()

best_mse

0.05250996

### 3 Classify iris data by using Deep Neural Network with TensorFlow

See [dnn_iris.py](dnn_iris.py)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feature, label, random_state=42, test_size=0.2)